In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [3]:
data = pd.read_csv('cleaned_data.csv')
data.head(10)

,Unnamed: 0,album,artist,content,label,score,Month,Day,Year,Genre1,Genre2,Genre3,Genre4
0,0,El Mal Querer,Rosalía,"global bass, the internet-centric underground ...",Sony,8.8,November,8,2018,Pop/R&B,NaN,NaN,NaN
1,1,You Won’t Get What You Want,Daughters,you know how life goes: one day you’re making ...,Ipecac,8.0,November,9,2018,Rock,NaN,NaN,NaN
2,2,Wham! Bang! Pow! Let’s Rock Out!,Art Brut,"on ,’s 2005 debut, ,, eddie argos made gaming ...",Alcopop!,7.0,November,24,2018,Rock,NaN,NaN,NaN
3,3,"4AM Mulholland EP,After 4AM EP","Amerie,Amerie",stung by file-sharing losses and the decline o...,"FEENIIX RISING ENTERTAINMENT, FEENIIX RISING ...","7.2,7.6",October,26,2018,Pop/R&B,NaN,NaN,NaN
4,4,Temporary etc.,Booker Stardrum,it might seem counterintuitive for a drummer t...,NNA Tapes,7.7,December,20,2018,Experimental,NaN,NaN,NaN
5,6,Moondance,Van Morrison,"“will you shut the fuck up and listen?” , aske...",Warner Bros.,8.4,November,25,2018,Rock,NaN,NaN,NaN
6,7,Ghost Box (Expanded),SUSS,if you want to approximate the hypothetical es...,Northern Spy,7.6,November,26,2018,Experimental,NaN,NaN,NaN
7,8,Poison,Swizz Beatz,“it’s a difference between a beatmaker and a p...,Epic Records,7.1,November,9,2018,Rap,NaN,NaN,NaN
8,9,Lost Time Behind the Moon,Scott Hirsch,’s long musical journey has come full circle. ...,Scissor Tail,7.5,December,19,2018,Electronic,NaN,NaN,NaN
9,10,Acrylic,Leikeli47,"no one ever sees , unmasked. the omnipresent b...","Hardcover,RCA",7.6,November,26,2018,Rap,NaN,NaN,NaN


In [4]:
data = data.loc[data['Genre1']== 'Rock']
data

,Unnamed: 0,album,artist,content,label,score,Month,Day,Year,Genre1,Genre2,Genre3,Genre4
1,1,You Won’t Get What You Want,Daughters,you know how life goes: one day you’re making ...,Ipecac,8.0,November,9,2018,Rock,NaN,NaN,NaN
2,2,Wham! Bang! Pow! Let’s Rock Out!,Art Brut,"on ,’s 2005 debut, ,, eddie argos made gaming ...",Alcopop!,7.0,November,24,2018,Rock,NaN,NaN,NaN
5,6,Moondance,Van Morrison,"“will you shut the fuck up and listen?” , aske...",Warner Bros.,8.4,November,25,2018,Rock,NaN,NaN,NaN
14,15,Atardecer,Friends of Dean Martinez,the moment i saw the packaging i knew somethin...,"Knitting Factory,Knitting Factory",8.3,March,23,1999,Rock,NaN,NaN,NaN
20,21,Seven More Minutes,The Rentals,"""i don't think this relationship is going anyw...","Reprise,Maverick,Reprise,Maverick",5.9,April,13,1999,Rock,NaN,NaN,NaN
25,26,The Sound of Music: An Unfinished Symphony in ...,Jad Fair,"many years ago, i picked up bongwater's ,. i k...","Shimmy Disc,Shimmy Disc",3.7,April,20,1999,Rock,NaN,NaN,NaN
27,28,The Hush,Texas,england's texas sculpts studio-perfect summer ...,"Interscope,Interscope",4.0,May,18,1999,Rock,NaN,NaN,NaN
29,30,Agaetis Byrjun,Sigur Rós,icelandic lore tells of the hidden people who ...,Smekkleysa,9.4,June,1,1999,Rock,NaN,NaN,NaN
32,33,The Only Reason I Feel Secure EP,Pedro the Lion,i was going through the magazine rack at local...,"Made in Mexico,Made in Mexico",8.4,June,1,1999,Rock,NaN,NaN,NaN
37,38,EEVIAC,Man or Astro-Man?,"good evening, earthlings. the dreaded planet s...","Touch and Go,Touch and Go",5.9,April,20,1999,Rock,NaN,NaN,NaN


In [5]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data['content']))

print(data_words[:1])

[['you', 'know', 'how', 'life', 'goes', 'one', 'day', 'you', 're', 'making', 'songs', 'like', 'selling', 'with', 'vagina', 'bat', 'flying', 'above', 'your', 'band', 'name', 'and', 'years', 'go', 'by', 'and', 'suddenly', 'everyone', 'getting', 'married', 'and', 'having', 'kids', 'after', 'all', 'the', 'actual', 'blood', 'piss', 'and', 'sweat', 'that', 'went', 'into', 'mocking', 'indie', 'culture', 'and', 'convention', 'your', 'friends', 'have', 'somehow', 'become', 'craft', 'brewers', 'and', 'commercial', 'music', 'writers', 'rhode', 'island', 'hardcore', 'misanthropes', 'never', 'seemed', 'the', 'sort', 'of', 'band', 'compatible', 'with', 'the', 'demands', 'of', 'adulthood', 'when', 'they', 'broke', 'up', 'in', 'due', 'to', 'extreme', 'internal', 'duress', 'there', 'was', 'little', 'reason', 'to', 'believe', 'some', 'newfound', 'cachet', 'changing', 'times', 'or', 'popular', 'demand', 'would', 'ever', 'coax', 'their', 'wildly', 'abrasive', 'music', 'back', 'into', 'action', 'they', 'ac

In [6]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['you', 'know', 'how', 'life', 'goes', 'one', 'day', 'you', 're', 'making', 'songs', 'like', 'selling', 'with', 'vagina', 'bat', 'flying', 'above', 'your', 'band', 'name', 'and', 'years', 'go', 'by', 'and', 'suddenly', 'everyone', 'getting', 'married', 'and', 'having', 'kids', 'after', 'all', 'the', 'actual', 'blood', 'piss', 'and', 'sweat', 'that', 'went', 'into', 'mocking', 'indie', 'culture', 'and', 'convention', 'your', 'friends', 'have', 'somehow', 'become', 'craft', 'brewers', 'and', 'commercial', 'music', 'writers', 'rhode_island', 'hardcore', 'misanthropes', 'never', 'seemed', 'the', 'sort', 'of', 'band', 'compatible', 'with', 'the', 'demands', 'of', 'adulthood', 'when', 'they', 'broke', 'up', 'in', 'due', 'to', 'extreme', 'internal', 'duress', 'there', 'was', 'little', 'reason', 'to', 'believe', 'some', 'newfound', 'cachet', 'changing', 'times', 'or', 'popular', 'demand', 'would', 'ever', 'coax', 'their', 'wildly', 'abrasive', 'music', 'back', 'into', 'action', 'they', 'actual

In [7]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [8]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_lg')

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['know', 'life', 'go', 'day', 'make', 'song', 'sell', 'vagina', 'bat', 'fly', 'band', 'name', 'year', 'go', 'suddenly', 'everyone', 'get', 'married', 'kid', 'actual', 'blood', 'piss', 'sweat', 'go', 'mock', 'indie', 'culture', 'convention', 'friend', 'somehow', 'become', 'craft', 'brewer', 'commercial', 'music', 'writer', 'rhode_island', 'hardcore', 'misanthrope', 'never', 'seem', 'sort', 'band', 'compatible', 'demand', 'adulthood', 'break', 'due', 'extreme', 'internal', 'duress', 'little', 'reason', 'believe', 'newfound', 'cachet', 'change', 'time', 'popular', 'demand', 'would', 'ever', 'coax', 'wildly', 'abrasive', 'music', 'back', 'action', 'actually', 'trick', 'reunite', 'anyway', 'organizing', 'principle', 'plan', 'linger', 'contempt', 'humanity', 'say', 'way', 'title', 'fourth', 'album', 'first', 'year', 'really', 'mean', 'get', 'deserve', 'daughter', 'work', 'provide', 'instantaneous', 'absurdist', 'release', 'string', 'minute', 'song', 'pummel', 'flee', 'minute', 'almost', 'lo

In [9]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

#filter extreme values in dictionary
id2word.filter_extremes(no_below=15, no_above=0.2, keep_n=50000)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 2), (29, 1), (30, 1), (31, 2), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 2), (40, 1), (41, 1), (42, 1), (43, 1), (44, 2), (45, 5), (46, 1), (47, 1), (48, 1), (49, 2), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 2), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1), (103, 1), (104, 1), (105, 1), (106, 1), (107, 1), (108, 1), (109, 1), (110, 1)

In [10]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [11]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.009*"project" + 0.009*"ep" + 0.008*"musician" + 0.006*"synth" + '
  '0.006*"artist" + 0.006*"film" + 0.006*"sometimes" + 0.005*"jazz" + '
  '0.005*"soundtrack" + 0.005*"duo"'),
 (1,
  '0.008*"space" + 0.007*"noise" + 0.006*"effect" + 0.006*"tone" + '
  '0.006*"rhythm" + 0.006*"piece" + 0.006*"instrumental" + 0.005*"shift" + '
  '0.005*"heavy" + 0.005*"synth"'),
 (2,
  '0.007*"blue" + 0.006*"dark" + 0.006*"country" + 0.006*"arrangement" + '
  '0.005*"singer_songwriter" + 0.005*"city" + 0.004*"deep" + 0.004*"sun" + '
  '0.004*"night" + 0.004*"rich"'),
 (3,
  '0.016*"smith" + 0.010*"art" + 0.010*"blur" + 0.008*"post_punk" + '
  '0.007*"grunge" + 0.006*"johnson" + 0.006*"underground" + 0.005*"future" + '
  '0.005*"morrison" + 0.005*"swan"'),
 (4,
  '0.011*"roll" + 0.010*"american" + 0.008*"white" + 0.008*"blue" + '
  '0.007*"woman" + 0.007*"spirit" + 0.007*"soul" + 0.006*"new_york" + '
  '0.006*"political" + 0.006*"artist"'),
 (5,
  '0.023*"version" + 0.018*"original" + 0.015*"la

In [12]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.727015087253198

Coherence Score:  0.35002048547461995


In [13]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.052329 -0.028254       1        1  19.360620
7     -0.097366 -0.113647       2        1  15.349197
9     -0.088227  0.057070       3        1  14.103945
1     -0.143781  0.078875       4        1  12.882529
0     -0.178599 -0.052760       5        1  10.121052
2     -0.180314  0.157895       6        1   8.140606
4      0.144158 -0.113441       7        1   7.360563
5      0.089393 -0.278851       8        1   5.811819
6      0.218661  0.292136       9        1   3.887496
3      0.288403  0.000977      10        1   2.982173, topic_info=     Category         Freq         Term        Total  loglift  logprob
712   Default  2091.000000      version  2091.000000  30.0000  30.0000
1493  Default  1603.000000     original  1603.000000  29.0000  29.0000
258   Default  2235.000000        later  2235.000000  28.0000  28.0000
44    Default  1140.000000        dance  1140.000000  27.0000  27.0000
1391  Default  1383.000000    recording  1383.000000  26.0000  26.0000
532   Default  1425.000000     material  1425.000000  25.0000  25.0000
1017  Default  1882.000000         blue  1882.000000  24.0000  24.0000
2922  Default  2313.000000        synth  2313.000000  23.0000  23.0000
359   Default  2109.000000       career  2109.000000  22.0000  22.0000
631   Default  1963.000000         self  1963.000000  21.0000  21.0000
3625  Default  1172.000000       groove  1172.000000  20.0000  20.0000
568   Default  1301.000000  performance  1301.000000  19.0000  19.0000
795   Default  1757.000000      project  1757.000000  18.0000  18.0000
691   Default  1659.000000         tour  1659.000000  17.0000  17.0000
623   Default  1609.000000         roll  1609.000000  16.0000  16.0000
131   Default  1057.000000       scream  1057.000000  15.0000  15.0000
1618  Default   906.000000         funk   906.000000  14.0000  14.0000
1958  Default  1797.000000           ep  1797.000000  13.0000  13.0000
656   Default  1827.000000        space  1827.000000  12.0000  12.0000
206   Default  1476.000000        death  1476.000000  11.0000  11.0000
1091  Default  1407.000000        noise  1407.000000  10.0000  10.0000
634   Default   994.000000      session   994.000000   9.0000   9.0000
1469  Default  1291.000000   collection  1291.000000   8.0000   8.0000
1149  Default  1086.000000     american  1086.000000   7.0000   7.0000
3158  Default  1634.000000        woman  1634.000000   6.0000   6.0000
9     Default  1439.000000          art  1439.000000   5.0000   5.0000
2640  Default   844.000000       garage   844.000000   4.0000   4.0000
1331  Default  1772.000000    influence  1772.000000   3.0000   3.0000
1194  Default  1745.000000        maybe  1745.000000   2.0000   2.0000
7539  Default   730.000000        smith   730.000000   1.0000   1.0000
...       ...          ...          ...          ...      ...      ...
2876  Topic10   141.428574        oasis   142.209991   3.5070  -5.7802
6539  Topic10   136.662201     anderson   137.443665   3.5068  -5.8145
2802  Topic10   130.612564      bookend   131.394135   3.5066  -5.8598
6804  Topic10   131.013504          dna   131.803940   3.5065  -5.8567
0     Topic10   126.387032     abrasive   127.168556   3.5064  -5.8927
2633  Topic10   125.652359         cure   126.433960   3.5063  -5.8985
7768  Topic10   125.297401        taunt   126.079033   3.5063  -5.9013
4887  Topic10   121.985329      chapter   122.766945   3.5061  -5.9281
2854  Topic10   121.792809         liar   122.574280   3.5061  -5.9297
5381  Topic10   121.704224    catalogue   122.485756   3.5061  -5.9304
7312  Topic10   121.519707      madness   122.301392   3.5061  -5.9320
4532  Topic10   120.192192    suffocate   120.973846   3.5060  -5.9429
2675  Topic10   215.889648      britpop   228.046524   3.4577  -5.3573
2561  Topic10   313.550659       grunge   395.390869   3.2806  -4.9841
1140  Topic10   171.024780        vital   2